# Gaussian Mixture Model in PyMC3

In [1]:
from timeit import default_timer as timer
import numpy as np
import pickle

import pymc3 as pm

from utils import generate_datasets, SEED

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1. Data

In [2]:
def load_datasets(filename='gmm_6k.pkl', n=3):
    """
    :param filename: name of the pickle file 
    :param n: number of datasets to read (defults to 3)
    :return: list of loaded datasets in dict format
    """
    datasets = []
        with open(filename, 'rb') as f:
            for i in range(n):
                dataset = pickle.load(f)
                datasets.append(dataset)      
    return datasets

## 2. Model

In [3]:
def pymc3_model(K, y):
    model = pm.Model()

      with model:
        w = pm.Dirichlet('w', np.ones(K))

        mu = pm.Normal('mu', 0., 10., shape=[K])
        sigma = pm.InverseGamma('sigma', 1., 1., shape=[K])

        y_obs = pm.NormalMixture('y_obs', w, mu, sd=sigma, observed=y)
    return model

## 2. Inference

In [4]:
def pymc3_nuts(filename='gmm_6k.pkl', n=3):
  datasets = load_datasets(filename, n)
  
  for dataset in datasets:
    print('...')
    model = pymc3_model(dataset['K'], dataset['y'])
    with model:
      start = timer()
      trace = pm.sample(3000, tune=2000, random_seed = dataset['seed'], chains=4)
      end = timer()
    trace_thin = trace[::10]
    
    mu_ = np.array(trace_thin.get_values('mu', combine=False)).mean(axis=0)
    sigma_ = np.array(trace_thin.get_values('sigma', combine=False)).mean(axis=0)
    w_ = np.array(trace_thin.get_values('w', combine=False)).mean(axis=0)

    divergent = trace_thin['diverging'].nonzero()[0].size
    results = {'mu': mu_, 'sigma': sigma_, 'w': w_, 'seed': dataset['seed'], 
               'iters': 5000, 'warmup': 2000, 'time': end-start, 'thin': 10, 'divergences': divergent}
    
    with open('results/pymc3/nuts_{}k_{}.pkl'.format(dataset['K'], dataset['seed']), 'wb') as f:
      pickle.dump(results, f)
  print('Done')

In [ ]:
pymc3_nuts(filename='gmm_3k.pkl')

In [ ]:
pymc3_nuts(filename='gmm_6k.pkl')

### ADVI

In [6]:
def pymc3_vi(filename='gmm_6k.pkl', n=3):
  """
  Runs PyMC3's ADVI algorithm (meanfield approximation) for each seed
  """
  datasets = load_datasets(filename, n)
  for dataset in datasets:
    model = pymc3_model(dataset['K'], dataset['y'])
    iters = np.linspace(500, 50000, 10).astype(int)
    for n in iters:
      with model:
        start = timer()
        advi_fit = pm.fit(n=n, random_seed=dataset['seed'], callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute', tolerance=0.000001)])
        end = timer()
        print('Time: ', end-start)
        trace = advi_fit.sample(draws=1000)
        mu_ = np.array(trace.get_values('mu'))
        sigma_ = np.array(trace.get_values('sigma'))
        w_ = np.array(trace.get_values('w'))
                      
        results = {'iters': n, 'tol': 0.000001, 'time': end-start, 'mu': mu_, 'sigma': sigma_, 'w': w_}
        pickle.dump(results, open('results/pymc3/vi_{}k_{}.pkl'.format(dataset['K'], dataset['seed']), 'ab'))
    print('Done')

In [ ]:
pymc3_vi(filename='gmm_3k.pkl', n=3)

In [ ]:
pymc3_vi(filename='gmm_6k.pkl', n=3)

## References

[1] Rochford, Austin [Marginalized Gaussian mixture model](https://docs.pymc.io/notebooks/marginalized_gaussian_mixture_model.html)